PACKAGE INSTALLATIONS AND IMPORTS

In [6]:
import pandas as pd

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
from sklearn.model_selection import train_test_split

DATA SET

In [12]:
# Path to the dataset.Device specific
dataset_path = '/Users/unnimaya/Documents/Projectexperiments/AIRLINEDATASET/usairlinetweets.csv'
df_airline = pd.read_csv(dataset_path)#load the dataset
print(df_airline.head())

             tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnar

In [13]:
df = df_airline[["airline_sentiment", "text"]]
df

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [14]:
df["airline_sentiment"].value_counts()
#checking the class distribution

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [15]:
#Performing test-train split
label_column = 'airline_sentiment'
# Taking 200 data points from each class for the test set
test_df_class_1 = df[df[label_column] == 'negative'].sample(n=200, random_state=42)
test_df_class_2 = df[df[label_column] == 'neutral'].sample(n=200, random_state=42)
test_df_class_3 = df[df[label_column] == 'positive'].sample(n=200, random_state=42)
# Concatenate the sampled data frames to create the test set
test_df_unmodified = pd.concat([test_df_class_1, test_df_class_2, test_df_class_3])# concatanating to form test data
train_df_unmodified = df.drop(test_df_unmodified.index)
#class distribution
print("Test set class distribution:\n", test_df_unmodified[label_column].value_counts())
print("Train set class distribution:\n", train_df_unmodified[label_column].value_counts())


Test set class distribution:
 airline_sentiment
negative    200
neutral     200
positive    200
Name: count, dtype: int64
Train set class distribution:
 airline_sentiment
negative    8978
neutral     2899
positive    2163
Name: count, dtype: int64


In [16]:
test_df_unmodified.index

Index([ 1151,  9111,  3326, 10891, 11322,  3709,  5261, 10060, 12446,  3089,
       ...
       11705, 13422,  2716,  3071, 13504,  4489,  8271, 11795,  7294,  8203],
      dtype='int64', length=600)

In [17]:
test_df_class_1.index

Index([ 1151,  9111,  3326, 10891, 11322,  3709,  5261, 10060, 12446,  3089,
       ...
       13324, 11935,  9346, 11781,  3022, 10759, 11149,  7523, 12754,   667],
      dtype='int64', length=200)

In [18]:
train_df_unmodified.shape

(14040, 2)

In [19]:
test_df_unmodified.shape

(600, 2)

In [20]:
df.shape

(14640, 2)

In [21]:
#Cleaning the data
lemmatizer = WordNetLemmatizer()#initialize lemmatizer
stop_words = set(stopwords.words('english'))#initialize stopwords

def text_cleaning(text):
    text = text.lower()#converting the text to lowercase
    text = re.sub(r'[^\w\s]', '', text)#punctuation removal using regular expression
    text = re.sub(r'\d+', '', text)#number removal
    
    tokens = word_tokenize(text) #tokenising the text
    tokens = [word_lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]#stop word removal an lemmatization
    cleaned_text = ' '.join(tokens) #joins the tokens to form cleaned sentence   
    return cleaned_text

In [22]:
%%time
#cleaning the train data
train_df_unmodified['text'] = train_df_unmodified['text'].apply(text_cleaning)

CPU times: user 2min 33s, sys: 3.54 s, total: 2min 36s
Wall time: 2min 42s


In [23]:
train_df_unmodified

,airline_sentiment,text
1,positive,virginamerica plus youve added commercial expe...
2,neutral,virginamerica didnt today must mean need take ...
3,negative,virginamerica really aggressive blast obnoxiou...
4,negative,virginamerica really big bad thing
5,negative,virginamerica seriously would pay flight seat ...
...,...,...
14635,positive,americanair thank got different flight chicago
14636,negative,americanair leaving minute late flight warning...
14637,neutral,americanair please bring american airline blac...
14638,negative,americanair money change flight dont answer ph...


In [24]:
%%time
#cleaning the test data
test_df_unmodified['text'] = test_df_unmodified['text'].apply(text_cleaning)

CPU times: user 5.95 s, sys: 111 ms, total: 6.06 s
Wall time: 6.11 s


In [25]:
test_df_unmodified

,airline_sentiment,text
1151,negative,united gate c iad u released passenger board w...
9111,negative,usairways connecting charlotte houston mechani...
3326,negative,united installed working kicked hour wouldnt l...
10891,negative,usairways flight fll told take train back pbi ...
11322,negative,usairways loosing lot business using barclays ...
...,...,...
4489,positive,southwestair left ipad seat pocket filed lost ...
8271,positive,jetblue thatd nice hoping rack enough mile tak...
11795,positive,usairways customer service finest
7294,positive,jetblue awesome


UNDERSAMPLING

In [26]:
%%time
from sklearn.utils import resample #importing resample function from sklearn

class_counts = train_df_unmodified['airline_sentiment'].value_counts()#Getting the number of data points in each class.
min_class_count = class_counts.min()  # Getting the number of samples in the minority class
negative_df = train_df_unmodified[train_df_unmodified['airline_sentiment'] == 'negative']#seperating each class to dataframes
neutral_df = train_df_unmodified[train_df_unmodified['airline_sentiment'] == 'neutral']
positive_df = train_df_unmodified[train_df_unmodified['airline_sentiment'] == 'positive']

#Here we are undersampling the negative and neutral classes to match the  sample size of minority class(positive) 
                                    replace=False,     # sampling without replacement
                                    n_samples=min_class_count,     #  giving min count as minority class count
                                    random_state=42)  # reproducible results

neutral_df_undersampled = resample(neutral_df,
                                   replace=False,     # sample without replacement
                                   n_samples=min_class_count,    #  giving min count as minority class count
                                   random_state=42)  # reproducible results

undersampled_df = pd.concat([negative_df_undersampled, neutral_df_undersampled, positive_df])#combining them to make an undersampled data frame
undersampled_df = undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)#shuffling the undersampled dataframe

print("Undersampled DataFrame:")
print(undersampled_df['airline_sentiment'].value_counts())
#From now on train data= undersampled df, test data= test_df_unmodified

Undersampled DataFrame:
airline_sentiment
neutral     2163
positive    2163
negative    2163
Name: count, dtype: int64
CPU times: user 9.82 ms, sys: 4.91 ms, total: 14.7 ms
Wall time: 15.2 ms


VADER

In [ ]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [27]:

analyzer = SentimentIntensityAnalyzer()  # Sentiment analyzer  initializing
#Function to obtain the sentiment scores using VADER
def obtain_sentiment_scores(texts):
    scores = []
    for text in texts:
        score = analyzer.polarity_scores(text)#calculating polarity scores
        scores.append(score['compound'])  # Using the compound score
    return scores

def obtain_binary_predictions(scores):#Here, we are getting the binary prediction score for auc calculation
    predictions = []
    for score in scores:
        if score >= 0.05:
            predictions.append('positive')
        elif score <= -0.05:
            predictions.append('negative')
        else:
            predictions.append('neutral')
    return predictions #return the scores


In [28]:
%%time

sentiment_scores_test = obtain_sentiment_scores(undersampled_df['text'])#calling the obtain_sentiment_scores function
y_predicted_test = obtain_binary_predictions(sentiment_scores_test)# getting the predictions

CPU times: user 189 ms, sys: 3.98 ms, total: 193 ms
Wall time: 198 ms


In [35]:
%%time
# Convert labels to binary for each sentiment class
#for test set
y_test_binary_positive = (undersampled_df['airline_sentiment'] == 'positive').astype(int)
y_test_binary_negative = (undersampled_df['airline_sentiment'] == 'negative').astype(int)
y_test_binary_neutral = (undersampled_df['airline_sentiment'] == 'neutral').astype(int)
#for the predictions
y_pred_binary_positive = (pd.Series(y_predicted_test) == 'positive').astype(int)
y_pred_binary_negative = (pd.Series(y_predicted_test) == 'negative').astype(int)
y_pred_binary_neutral = (pd.Series(y_predicted_test) == 'neutral').astype(int)

CPU times: user 7.39 ms, sys: 13.4 ms, total: 20.8 ms
Wall time: 318 ms


In [40]:
%%time
#f1 score calculation
f1_score_positive_vader = f1_score(y_test_binary_positive, y_pred_binary_positive)#For positive
print("Positive F1 Score:", f1_score_positive)
f1_score_negative_vader = f1_score(y_test_binary_negative, y_pred_binary_negative)#For negative
print("Negative F1 Score:", f1_score_negative)
f1_score_neutral_vader = f1_score(y_test_binary_neutral, y_pred_binary_neutral)#For neutral
print("Neutral F1 Score:", f1_score_neutral)
average_f1_score_vader = (f1_score_positive_vader + f1_score_negative_vader + f1_score_neutral_vader) / 3
print("Average auc score:",average_f1_score_vader)

Positive F1 Score: 0.6125838390290642
Negative F1 Score: 0.481994459833795
Neutral F1 Score: 0.41822901644072685
Average auc score: 0.5042691051011953
CPU times: user 11.3 ms, sys: 11.8 ms, total: 23.2 ms
Wall time: 78.2 ms


In [39]:
%%time
# Calculating AUC score
auc_score_positive = roc_auc_score(y_test_binary_positive, sentiment_scores_test)# for positive class
print("Positive AUC Score:", auc_score_positive)
auc_score_negative = roc_auc_score(y_test_binary_negative, sentiment_scores_test)# for negative class
print("Negative AUC Score:", auc_score_negative)
auc_score_neutral = roc_auc_score(y_test_binary_neutral, sentiment_scores_test)#for neutral class
print("Neutral AUC Score:", auc_score_neutral)

Positive AUC Score: 0.7948918782644865
Negative AUC Score: 0.2770805026066731
Neutral AUC Score: 0.4280276191288405
Average auc score: 0.5000000000000001
CPU times: user 15.5 ms, sys: 13.8 ms, total: 29.2 ms
Wall time: 58 ms


In [127]:
%%time
# Calculating accuracy score for the test set
accuracy = accuracy_score(undersampled_df['airline_sentiment'], y_predicted_test)#overall accuracy
print("Overall Accuracy:", accuracy)
accuracy_positive = accuracy_score( y_test_binary_positive, y_pred_binary_positive )# accuracy for positive class
print("Positive Accuracy:", accuracy_positive)
accuracy_negative = accuracy_score( y_test_binary_negative, y_pred_binary_negative )# accuracy for negative class
print("Negative Accuracy:", accuracy_negative)
accuracy_neutral = accuracy_score( y_test_binary_neutral, y_pred_binary_neutral )#accuracy for neutral class
print("Neutral Accuracy:", accuracy_neutral)

Overall Accuracy: 0.5279704114655571
Positive Accuracy: 0.626136538757898
Negative Accuracy: 0.7406380027739251
Neutral Accuracy: 0.6891662813992911
CPU times: user 17.6 ms, sys: 3.34 ms, total: 20.9 ms
Wall time: 19.6 ms


AFINN STARTS HERE

In [42]:
from sklearn.metrics import accuracy_score
from afinn import Afinn
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize

In [43]:
%%time
afinn = Afinn()#Initialising AFINN function
def get_afinn_scores(text):
    score = afinn.score(text)
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

CPU times: user 46.2 ms, sys: 12 ms, total: 58.1 ms
Wall time: 162 ms


In [44]:
#Getting the scores my applying the get_afinn_scores function
undersampled_df['predicted_sentiment'] = undersampled_df['text'].apply(get_afinn_scores)

In [45]:
%%time
#Prediction 
df_test = undersampled_df['text'].to_frame()
df_test['predicted_sentiment'] = df_test['text'].apply(get_afinn_scores)#getting the predictions
# Accuracy calculation
accuracy_afinn = accuracy_score(undersampled_df['airline_sentiment'], df_test['predicted_sentiment'])
print(f"Accuracy: {accuracy_afinn}")

Accuracy: 0.5495453844968408
CPU times: user 1.85 s, sys: 121 ms, total: 1.97 s
Wall time: 2.35 s


In [133]:
%%time
#Binarising for auc calculation
y_test_bin = label_binarize(undersampled_df['airline_sentiment'], classes=['negative', 'neutral', 'positive'])#test
y_pred_bin = label_binarize(df_test['predicted_sentiment'], classes=['negative', 'neutral', 'positive'])#prediction
#auc calculation
try:
    auc_afinn = roc_auc_score(y_test_bin, y_pred_bin, average='macro', multi_class='ovo')
    print(f"AUC: {auc_afinn}")
except ValueError:
    print("AUC cannot be computed for single class predictions.")

AUC: 0.6621590383726307
CPU times: user 20.1 ms, sys: 4.62 ms, total: 24.8 ms
Wall time: 23.6 ms


In [46]:
%%time
#Calculating f1score for afinn in undersampled data
f1 = f1_score(undersampled_df['airline_sentiment'], df_test['predicted_sentiment'], average=None, labels=['positive', 'negative', 'neutral'])
# calculating average f1score
f1_average_afinn = f1_score(undersampled_df['airline_sentiment'], df_test['predicted_sentiment'], average='macro', labels=['positive', 'negative', 'neutral'])

print("F1 Scores:")
print(f"F1 Score for Positive: {f1[0]}")#class positive
print(f"F1 Score for Negative: {f1[1]}")#class negative
print(f"F1 Score for Neutral: {f1[2]}")#class neutral
print(f"Average F1 Score afinn undersampled: {f1_average_afinn}")

F1 Scores:
F1 Score for Positive: 0.6340117281821318
F1 Score for Negative: 0.5246968352558414
F1 Score for Neutral: 0.44274809160305345
Average F1 Score afinn undersampled: 0.5338188850136756
CPU times: user 52.2 ms, sys: 8.23 ms, total: 60.4 ms
Wall time: 88.3 ms


Textblob starts here

In [48]:

from textblob import TextBlob
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize


In [49]:
# Function to compute TextBlob sentiment scores
def get_textblob_scores(text):
    analysis = TextBlob(text)
    score = analysis.sentiment.polarity#calculating polarity scores for textblob
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

In [138]:
%%time
# getting the predictions
undersampled_df['predicted_sentiment'] = undersampled_df['text'].apply(get_textblob_scores)

CPU times: user 401 ms, sys: 19.3 ms, total: 421 ms
Wall time: 436 ms


In [51]:
%%time
#prediction calculation
df_test = undersampled_df['text'].to_frame()
df_test['predicted_sentiment_textblob'] = df_test['text'].apply(get_textblob_scores)
# Accuracy calculation for textBlob
accuracy_textblob = accuracy_score(undersampled_df['airline_sentiment'], df_test['predicted_sentiment_textblob'])
print(f"Accuracy: {accuracy_textblob}")

Accuracy: 0.5515487748497457
CPU times: user 564 ms, sys: 19.2 ms, total: 584 ms
Wall time: 613 ms


In [54]:
%%time
#auc for textblob on undersampled data
y_test_bin_textblob = label_binarize(undersampled_df['airline_sentiment'], classes=['negative', 'neutral', 'positive'])[:, 2]  # 1 for 'positive', 0 for others
y_pred_bin_textblob = label_binarize(df_test['predicted_sentiment_textblob'], classes=['negative', 'neutral', 'positive'])[:, 2]
#multiclass auc calculation 
try:
    auc_textblob = roc_auc_score(y_test_bin_textblob, y_pred_bin_textblob)
    print(f"AUC: {auc_textblob}")
except ValueError:
    print("AUC calculation not possible for single class predictions.")

AUC: 0.7172907998150717
CPU times: user 14 ms, sys: 13.4 ms, total: 27.4 ms
Wall time: 41.6 ms


In [55]:
%%time
#getting f1score for textblob in undersampled data
f1 = f1_score(undersampled_df['airline_sentiment'], df_test['predicted_sentiment_textblob'], average=None, labels=['positive', 'negative', 'neutral'])
f1_average_textblob = f1_score(undersampled_df['airline_sentiment'], df_test['predicted_sentiment_textblob'], average='macro', labels=['positive', 'negative', 'neutral'])
print(f"F1 Score for Positive: {f1[0]}")
print(f"F1 Score for Negative: {f1[1]}")
print(f"F1 Score for Neutral: {f1[2]}")
print(f"Average f1score textblob: {f1_average_textblob}")


F1 Score for Positive: 0.6292847235981771
F1 Score for Negative: 0.45660964230171075
F1 Score for Neutral: 0.5330788804071247
Average f1score textblob: 0.5396577487690042
CPU times: user 49.9 ms, sys: 5.65 ms, total: 55.5 ms
Wall time: 79.6 ms


In [ ]:
ML METHODS-UNDERSAMPLED DATA

SELF SUPERVISED LEARNING


In [68]:
#importing packages 
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [71]:
%%time
#preprocessing the train data
nltk.download('vader_lexicon')
#initialize sentiment analyser
sid = SentimentIntensityAnalyzer()
train_df, val_df = train_test_split(undersampled_df, test_size=0.2, random_state=42)#taking 20 percent of train data as validation set
test_df= test_df_unmodified
# Obtaining the sentiment labels
def get_sentiment_label_SSL(text):
    scores = sid.polarity_scores(text)#Calculating polarity scores
    compound = scores['compound']#calculating compound scores
    if compound >= 0.5:
        return 'positive', compound
    elif compound <= -0.5:
        return 'negative', compound
    else:
        return 'neutral', compound

#Psedolabel using different confidence threshold for each class
def pseudo_labeling_vader(df, pos_threshold=0.8, neg_threshold=0.5, neutral_threshold=(0.1, -0.1)):
    pseudo_texts = []
    pseudo_labels = []#list for pseudolabels
    for text in df['text']:#iterate through the text and get sentiment label and confidence score for each
        sentiment, confidence = get_sentiment_label_SSL(text)
        #assigning pseudolabels according to the threshold
        if sentiment == 'positive' and confidence >= pos_threshold:
            pseudo_texts.append(text)#append the text to pseudo texts
            pseudo_labels.append(sentiment)# append the label for the text to pseudolabels
        elif sentiment == 'negative' and confidence <= -neg_threshold:
            pseudo_texts.append(text)
            pseudo_labels.append(sentiment)
        elif sentiment == 'neutral' and neutral_threshold[0] >= confidence >= neutral_threshold[1]:
            pseudo_texts.append(text)
            pseudo_labels.append(sentiment)
    return pseudo_texts, pseudo_labels

pseudo_texts, pseudo_labels = pseudo_labeling_vader(train_df)#getting pseudo labels

pseudo_df = pd.DataFrame({'text': pseudo_texts, 'airline_sentiment': pseudo_labels})#creating a pseudo labelled dataframe

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(pseudo_df['text'])#vectorising the text as part of preprocessing train data. 
X_val = vectorizer.transform(val_df['text'])

# Encoding the labels in train and validation set
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(pseudo_df['airline_sentiment'])
y_val = label_encoder.transform(val_df['airline_sentiment'])


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/unnimaya/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


CPU times: user 389 ms, sys: 23.6 ms, total: 412 ms
Wall time: 434 ms


In [72]:
%%time
#test data preprocessing
X_test = vectorizer.transform(test_df['text'])# Vectorization of data
y_test = label_encoder.transform(test_df['airline_sentiment'])#Encoding the labels

CPU times: user 14.2 ms, sys: 3.09 ms, total: 17.3 ms
Wall time: 17.7 ms


In [73]:
%%time 
#training
# Model fitting
svm_model = SVC(probability=True, kernel='linear')#support vector machine with linear kernel
svm_model.fit(X_train, y_train)

CPU times: user 1.02 s, sys: 20.5 ms, total: 1.04 s
Wall time: 1.06 s


SVC(kernel='linear', probability=True)

In [74]:
%%time
# Validation
val_predictions = svm_model.predict(X_val)#calculating the validation accurcay for SSL
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {val_accuracy}")

Validation Accuracy: 0.5184899845916795
CPU times: user 122 ms, sys: 3.28 ms, total: 126 ms
Wall time: 125 ms


In [75]:
%%time
# Testing
test_predictions = svm_model.predict(X_test)#predicting using SSL
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy}")#Calculating test accuracy for SSL

Test Accuracy: 0.5483333333333333
CPU times: user 76.6 ms, sys: 2.28 ms, total: 78.9 ms
Wall time: 77.1 ms


BAGGING WITH KNN

In [76]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [77]:
%%time
#defining test and train sets
X_train= undersampled_df['text']
y_train=undersampled_df['airline_sentiment']
X_test=test_df_unmodified['text']
y_test=test_df_unmodified['airline_sentiment']

CPU times: user 262 µs, sys: 1.06 ms, total: 1.32 ms
Wall time: 1.34 ms


In [78]:
%%time
def tokenize(text):
    return word_tokenize(text)#tokenizing

# Pipeline definition for vectorization and classifier
pipeline_bagging = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('bagging_knn', BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=3), n_estimators=10, random_state=42))
])

CPU times: user 76 µs, sys: 48 µs, total: 124 µs
Wall time: 129 µs


In [79]:
%%time
#training
pipeline_bagging.fit(X_train, y_train)#fitting the pipeline

/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 1min 2s, sys: 1.2 s, total: 1min 3s
Wall time: 1min 5s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x2827e0dc0>)),
                ('bagging_knn',
                 BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=3),
                                   random_state=42))])

In [80]:
%%time
#testing
y_pred_bagging = pipeline_bagging.predict(X_test)#predict bagging knn
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)#calculating accuracy of bagging knn
print("Bagging KNN Accuracy:", accuracy_bagging)

Bagging KNN Accuracy: 0.6033333333333334
CPU times: user 6.53 s, sys: 128 ms, total: 6.66 s
Wall time: 6.72 s


BOOSTING WTH DECISION TREES

In [81]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [82]:
%%time
X_train= undersampled_df['text']
y_train=undersampled_df['airline_sentiment']
X_test=test_df_unmodified['text']
y_test=test_df_unmodified['airline_sentiment']

CPU times: user 34 µs, sys: 142 µs, total: 176 µs
Wall time: 178 µs


In [83]:
def tokenize(text):
    return word_tokenize(text)

# Boosting with Decision Tree -pipeline deifnition
pipeline_boosting = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('boost_tree', AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3), n_estimators=50, random_state=42))#depth of the tree is 3
])

In [84]:
%%time
#training
pipeline_boosting.fit(X_train, y_train)#model fitting

/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


CPU times: user 1min 3s, sys: 941 ms, total: 1min 3s
Wall time: 1min 4s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x2827e31c0>)),
                ('boost_tree',
                 AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3),
                                    random_state=42))])

In [85]:
%%time
#testing
y_pred_boosting = pipeline_boosting.predict(X_test)#running predictions
accuracy_boosting = accuracy_score(y_test, y_pred_boosting)#accuracy score calculation
print("Boosting with Decision Tree Accuracy:", accuracy_boosting)

Boosting with Decision Tree Accuracy: 0.6733333333333333
CPU times: user 5.79 s, sys: 98.6 ms, total: 5.88 s
Wall time: 6.21 s


VOTING WITH LOGISTIC REGRESSION, SVM AND DECISION TREE

In [87]:
#ensemble model- voting using logistic regression, svc and decision tree
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/unnimaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [88]:
X_train= undersampled_df['text']
y_train=undersampled_df['airline_sentiment']
X_test=test_df_unmodified['text']
y_test=test_df_unmodified['airline_sentiment']

In [89]:
%%time
#Function for tokenizing the data.
def tokenize(text):
    return word_tokenize(text)
# Defining the individual models for voting
model1 = LogisticRegression(random_state=42, max_iter=1000)# the max iteration is given as 1000
model2 = SVC(probability=True, random_state=42)
model3 = DecisionTreeClassifier(random_state=42)
# Voting Classifier: voting category is soft 
voting_classifier = VotingClassifier(estimators=[
    ('lr', model1),#lr for logistic regression
    ('svc', model2),#svc for support vector machine
    ('dt', model3)#dt for decision tree
], voting='soft')

# Pipeline creation.
pipeline_voting = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('voting', voting_classifier)
])


CPU times: user 187 µs, sys: 1.07 ms, total: 1.26 ms
Wall time: 1.69 ms


In [90]:
%%time
# Train the pipeline
pipeline_voting.fit(X_train, y_train)#pipeline fitting

/Users/unnimaya/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 1min 22s, sys: 2.03 s, total: 1min 24s
Wall time: 1min 24s


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x2827e3130>)),
                ('voting',
                 VotingClassifier(estimators=[('lr',
                                               LogisticRegression(max_iter=1000,
                                                                  random_state=42)),
                                              ('svc',
                                               SVC(probability=True,
                                                   random_state=42)),
                                              ('dt',
                                               DecisionTreeClassifier(random_state=42))],
                                  voting='soft'))])

In [91]:
%%time
# Running predictions.
y_pred_voting = pipeline_voting.predict(X_test)
#Accuracy calculation for voting algorithm
accuracy_voting = accuracy_score(y_test, y_pred_voting)
print("Voting Classifier Accuracy:", accuracy_voting)

Voting Classifier Accuracy: 0.7183333333333334
CPU times: user 6.34 s, sys: 157 ms, total: 6.5 s
Wall time: 6.87 s
